In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [3]:
llm = ChatOpenAI()

In [4]:
prompt = ChatPromptTemplate.from_template("tell me about the nutritional value of {input}")

In [5]:
chain = LLMChain(llm=llm, prompt=prompt)
chain.predict(input="Pizza")

"Pizza is a popular and widely consumed food that consists of a round, flat base of dough topped with various ingredients such as tomato sauce, cheese, vegetables, meat, and spices. The nutritional value of pizza can vary depending on the type of crust, toppings, and portion size. \n\nHere is a breakdown of the nutritional components commonly found in a typical slice of pizza (approximately 14-inch, cheese pizza):\n\n1. Calories: A slice of cheese pizza typically contains around 250-300 calories, but this can vary based on the size of the slice and the amount of cheese used.\n\n2. Carbohydrates: Pizza is rich in carbohydrates due to its dough crust. A slice of pizza can provide around 30-40 grams of carbohydrates, primarily from refined flour used in the crust.\n\n3. Protein: Cheese pizza contains a moderate amount of protein, with one slice providing about 12-15 grams. This protein comes from the cheese and, if present, any meat or vegetable toppings.\n\n4. Fat: The fat content in piz

In [6]:
chain = prompt | llm
chain.invoke({"input": "Spaghetti"})

AIMessage(content="Spaghetti is a type of pasta made from wheat flour and water. The nutritional value of spaghetti can vary depending on the specific brand or recipe used, as well as the serving size. Here are some general nutritional facts about spaghetti:\n\n1. Calories: A one-cup serving of cooked spaghetti contains around 200-220 calories. This value can increase if you add sauces, cheese, or other toppings.\n\n2. Carbohydrates: Spaghetti is primarily composed of carbohydrates. One cup of cooked spaghetti provides approximately 40-45 grams of carbohydrates, which are an essential source of energy for the body.\n\n3. Protein: Spaghetti is not particularly high in protein. A one-cup serving typically contains around 7-8 grams of protein. To increase protein intake, you can pair spaghetti with protein-rich sources like lean meats, tofu, or legumes.\n\n4. Fiber: Whole wheat spaghetti contains more fiber than refined white spaghetti. On average, one cup of cooked whole wheat spaghetti 

### First pipe

In [9]:
from langchain.schema.output_parser import StrOutputParser

chain = prompt | llm | StrOutputParser()

In [10]:
chain = prompt | llm | StrOutputParser()
chain.invoke({"input": "Lasagna"})

"Lasagna is a traditional Italian dish made with layers of pasta, meat sauce, cheese, and sometimes vegetables. The nutritional value of lasagna can vary depending on the specific recipe and ingredients used, but here is a general overview:\n\n1. Carbohydrates: Lasagna is typically high in carbohydrates due to the pasta layers. Carbohydrates provide energy for the body.\n\n2. Proteins: The meat sauce in lasagna, usually made with ground beef or sausage, contributes to the protein content. Protein is essential for building and repairing tissues in the body.\n\n3. Fats: The cheese used in lasagna adds to the fat content. While some fats are necessary for the body, excessive consumption can contribute to weight gain and other health issues.\n\n4. Fiber: If vegetables like spinach, mushrooms, or zucchini are included in lasagna, it can provide additional fiber. Fiber aids in digestion, helps maintain bowel regularity, and can contribute to satiety.\n\n5. Vitamins and Minerals: Lasagna can 

In [ ]:
chain.invoke({"input": "A Pizza"})

In [ ]:
chain = prompt | llm | StrOutputParser()
chain.invoke({"input": "A Pizza"})

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "tell me 5 jokes about {input}"
    )

In [ ]:
chain = prompt | llm.bind(stop=["\n"]) | StrOutputParser()

In [ ]:
chain.invoke({"input": "pizzas"})

### OPENAI Functions

In [ ]:
functions = [
    {
      "name": "joke",
      "description": "A joke",
      "parameters": {
        "type": "object",
        "properties": {
          "setup": {
            "type": "string",
            "description": "The setup for the joke"
          },
          "punchline": {
            "type": "string",
            "description": "The punchline for the joke"
          }
        },
        "required": ["setup", "punchline"]
      }
    }
  ]

In [ ]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
chain = (
    prompt 
    | llm.bind(function_call={"name": "joke"}, functions= functions) 
    | JsonOutputFunctionsParser()
)

In [ ]:
chain.invoke(input={"input": "bears"})


### Working with vectorstores

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough


vectorstore = Chroma.from_texts(["Cats are typically 9.1 kg in weight.", 
                                 "Cats have retractable claws.", 
                                 "A group of cats is called a clowder.", 
                                 "Cats can rotate their ears 180 degrees.", 
                                 "The world's oldest cat lived to be 38 years old."], 
                                embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm 
    | StrOutputParser()
)

In [ ]:
chain.invoke("how old is the oldest cat?")

In [ ]:
from operator import itemgetter

In [ ]:
question = {"bla": "test", "x": "wuff"}
itemgetter("bla")


In [ ]:
get_bla = itemgetter("bla")
get_bla(question)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

# chain = {
#     "context": itemgetter("question") | retriever,
#     "question": itemgetter("question"),
#     "language": itemgetter("language")
# } | prompt | model | StrOutputParser()

chain = {
    "context": (lambda x: x["question"]) | retriever, 
    "question": (lambda x: x["question"]), 
    "language": (lambda x: x["language"])
} | prompt | llm | StrOutputParser()


In [ ]:
chain.invoke({"question": "how old is the oldest cat?", "language": "german"})

In [ ]:
!pip install duckduckgo-search

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

In [ ]:
template = """turn the following user input into a search query for a search engine:

{input}"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = prompt | llm | StrOutputParser() | search

In [ ]:
chain.invoke({"input": "whats the name of the oldest cat?"})

### Arbitrary functions

In [ ]:
from langchain.schema.runnable import RunnableLambda

def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain = {
    "a": itemgetter("foo") | RunnableLambda(length_function),
    "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")} | RunnableLambda(multiple_length_function)
} | prompt | llm | StrOutputParser()

In [ ]:
chain.invoke({"foo": "bar", "bar": "gah"})

### Interface

In [ ]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

In [ ]:
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="")

In [ ]:
chain.invoke({"topic": "bears"})

In [ ]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}])

In [ ]:
async for s in chain.astream({"topic": "bears"}):
    print(s.content, end="")
await chain.ainvoke({"topic": "bears"})
await chain.abatch([{"topic": "bears"}])